In [1]:
from sklearn.metrics import r2_score, make_scorer
from sklearn.linear_model import ElasticNet, HuberRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import KFold
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from scipy.stats import zscore
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pytorch_tabnet.tab_model import TabNetRegressor
import torch

In [2]:
x_test = pd.read_csv('X_GAIN_test.csv', header=0, index_col=0)
x_train = pd.read_csv('X_GAIN_train.csv', header=0, index_col=0)
y_train = pd.read_csv('y_train.csv', header=0, index_col=0)

In [ ]:
def display(X):
    print(pd.DataFrame(X, columns=x_train.columns))

In [ ]:
def preprocessing(X_train, X_test):
    imputer = KNNImputer()
    imputer.fit(np.concatenate((X_train, X_test)))
    return (imputer.transform(X_train), imputer.transform(X_test))

In [ ]:
def normalize(X_train, X_test):
    scaler = StandardScaler()
    concat = np.concatenate((X_train, X_test))
    scaler.fit(concat)
    return (scaler.transform(X_train), scaler.transform(X_test))

In [ ]:
def outlier_detection(X, y):
    clf = IsolationForest()
    clf.fit(X)

    outliers = []

    idx = 0

    for i in clf.predict(X):
        if i == -1:
            outliers.append(idx)
        idx += 1


    return (np.delete(X, outliers, 0), np.delete(y, outliers).reshape((-1, 1)))
    

In [ ]:
def feature_selection(model, X, y):
    raise NotImplementedError

In [27]:
def TabNetRegression(X_train, y_train):
    kf = KFold(shuffle=True)
    kf.get_n_splits(X_train, y_train)
    reg = TabNetRegressor(seed=42)
    for train_i, val_i in kf.split(X_train, y_train):
        X_trainkf, X_val = X_train[train_i], X_train[val_i]
        y_trainkf, y_val = y_train[train_i], y_train[val_i]
        reg.fit(
            X_train=X_trainkf, y_train=y_trainkf,
            eval_set=[(X_val, y_val)],
            drop_last=False,
            warm_start=True
        )


In [ ]:
def regression(X_train, y_train, X_test):
    reg = linear_model.Lasso(max_iter=100000)
    reg.fit(X_train, y_train)

    return reg.predict(X_test)

In [ ]:
X_processed_train, X_processed_test = x_train.to_numpy(), x_test.to_numpy()
y_train = y_train.to_numpy()

In [ ]:
X_processed_train, X_processed_test = preprocessing(X_processed_train, X_processed_test)


In [ ]:
X_processed_train, X_processed_test = normalize(X_processed_train, X_processed_test)

In [ ]:
X_processed_train, y_processed_train = outlier_detection(X_processed_train, y_train)


In [ ]:
results = regression(X_processed_train, y_processed_train, X_processed_test)

In [ ]:
asdf = pd.DataFrame(results)
asdf.index.name = 'id'
asdf.rename(columns={ 0: 'y' }, inplace=True)
asdf.to_csv("res.csv")